# **Project: MapGAN**
## Authors: Neill Shikada and Melanie Sharif
## Professor: Robin Burke
## Course: INFO5604

If you want to run this code locally using Anaconda, follow the instructions [here](https://research.google.com/colaboratory/local-runtimes.html):
1. Open Anaconda command prompt

2. Ensure that Jupyter has the proper extensions enabled
  `jupyter-serverextension enable jupyter_http_over_ws`

3. Run Jupyter Notebook and allow access to Google Colab with this code
  `jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'`
  
4. Click the dropdown in the top left of Google Colab Notebook, select `Connect to a local runtime`

5. Copy the `http://localhost:8888/?token=###`  link from the Anaconda command prompt

6. Paste into the Google Colab url space

7. It should connect! Then keep the Jupyter Notebook open the whole time you code as this is where your code is running locally

# Import Dependencies

1. Install any package that needs to be installed

In [ ]:
!pip install earthengine-api --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 244 kB 6.5 MB/s 
  Created wheel for earthengine-api: filename=earthengine_api-0.1.334-py3-none-any.whl size=274127 sha256=d24271646671b53aab7c7e14522175dbd805afdca811778c075ae9d10a752622
  Stored in directory: /root/.cache/pip/wheels/0a/d9/6b/1601dd4443ffced1345b91e6635c334a0c21f492a180c1ad6a
Successfully built earthengine-api
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 0.1.332
    Uninstalling earthengine-api-0.1.332:
      Successfully uninstalled earthengine-api-0.1.332


In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow as tf
print(tf.__version__)

2.11.0


2. Import all the necessary packages

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.callbacks import Callback

In [6]:
import os

In [1]:
import numpy as np

In [7]:
# Folium will be used for visualization purposes. It is a package that enables Google-Maps-like interactive maps
import folium
print(folium.__version__)

0.13.0


In [2]:
# Enable access to Google Earth Engine. If you are running this locally, you have to right click and open the link in a new tab (otherwise it'll come up with an error)
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvv69tglFq05GPcfaJ1nea6KiKWg6WdN6PYkot1Jc6xlYEv70_uxMVI

Successfully saved authorization token.


In [ ]:
# Your Earth Engine username.  This is used to import a classified image
# into your Earth Engine assets folder.
USER_NAME = 'nesh5910@colorado.edu'

# Cloud Storage bucket into which training, testing and prediction 
# datasets will be written.  You must be able to write into this bucket.
OUTPUT_BUCKET = 'mapgan'

# Use Landsat 8 surface reflectance data for predictors.
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Use these bands for prediction.
# The bands are different for each dataset in Earth Engine, but we are looking for the raw RGB values of the images
BANDS = ['B2', 'B3', 'B4']

# This is a trianing/testing dataset of points with known land cover labels.
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# The labels, consecutive integer indices starting from zero, are stored in
# this property, set on each point.
LABEL = 'landcover'
# Number of label values, i.e. number of classes in the classification.
N_CLASSES = 3

# These names are used to specify properties in the export of
# training/testing data and to define the mapping between names and data
# when reading into TensorFlow datasets.
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# File names for the training and testing datasets.  These TFRecord files
# will be exported from Earth Engine into the Cloud Storage bucket.
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# File name for the prediction (image) dataset.  The trained model will read
# this dataset and make predictions in each pixel.
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# The output path for the classified image (i.e. predictions) TFRecord file.
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# Export imagery in this region.
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# The name of the Earth Engine asset to be created by importing
# the classified image from the TFRecord file in Cloud Storage.
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'